# Automatización **Explorador** *DOC*

In [2]:
pip install python-docx

Note: you may need to restart the kernel to use updated packages.


In [3]:
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

In [4]:
document = Document()

# Agregar una imagen (similar a la imagen del logo en fpdf)
document.add_picture('tools/DatLogo.png', width=Inches(3))

# Agregar un título (similar al header en fpdf)
title = document.add_heading('EXPLORADOR DE PROPIEDADES', 0)
title.alignment = WD_ALIGN_PARAGRAPH.CENTER

# Agregar un subtítulo
subtitle = document.add_heading('Análisis de precios', 1)
subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER



# Agregar detalles de la propiedad
property_data = {
    'ubicacion': 'Circuito El Cerrito 140, Privadas de La Herradura, CP 42082, Pachuca de Soto, Hidalgo.',
    'valor': 3500000,
    'fecha': '26/06/2024'
}

# Agregar imagen de la propiedad
document.add_picture('tools/Propiedad.jpg', width=Inches(4))

# Configurar detalles de la propiedad
details = document.add_paragraph()
details.alignment = WD_ALIGN_PARAGRAPH.LEFT

# Ubicación
run = details.add_run('Ubicación de la propiedad: ')
run.font.bold = True
run = details.add_run(property_data['ubicacion'])

# Valor
details = document.add_paragraph()
run = details.add_run('Valor de la Propiedad: ')
run.font.bold = True
run = details.add_run(f"${property_data['valor']:,}")

# Fecha
details = document.add_paragraph()
run = details.add_run('Fecha de entrega: ')
run.font.bold = True
run = details.add_run(property_data['fecha'])

# Guardar el documento
document.save('Explorador.docx')


In [7]:
from docx import Document
from docx.shared import Inches, Pt, Cm
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import RGBColor
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

def set_cell_border(cell):
    """
    Set cell border - simplified version
    """
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()
    
    # Create right border element
    right = OxmlElement('w:right')
    right.set(qn('w:val'), 'single')
    right.set(qn('w:sz'), '4')
    right.set(qn('w:color'), '6495ED')
    
    # Create borders element
    borders = OxmlElement('w:tcBorders')
    borders.append(right)
    
    # Remove any existing borders
    for border in tcPr.findall('.//w:tcBorders'):
        tcPr.remove(border)
    
    # Add new borders
    tcPr.append(borders)

def create_property_document(logo_path, property_image_path, property_data):
    # Create document
    document = Document()
    
    # Set page margins
    sections = document.sections
    for section in sections:
        section.top_margin = Cm(2)
        section.bottom_margin = Cm(2)
        section.left_margin = Cm(2.54)
        section.right_margin = Cm(2.54)
    
    # Add logo
    logo_paragraph = document.add_paragraph()
    logo_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
    logo_run = logo_paragraph.add_run()
    logo_run.add_picture(logo_path, width=Inches(4.7))
    
    # Add main title
    title = document.add_paragraph()
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER
    title_run = title.add_run('EXPLORADOR DE PROPIEDADES')
    title_run.font.size = Pt(25)
    title_run.font.bold = True
    
    # Add subtitle
    subtitle = document.add_paragraph()
    subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER
    subtitle_run = subtitle.add_run('Análisis de precios')
    subtitle_run.font.size = Pt(19)
    subtitle_run.font.italic = True
    
    # Add space
    document.add_paragraph()
    
    # Create table for two-column layout
    table = document.add_table(rows=1, cols=2)
    table.autofit = False
    table.allow_autofit = False
    
    # Set column widths (60-40 split)
    table.columns[0].width = Inches(4.5)
    table.columns[1].width = Inches(3)
    
    # Get cells
    left_cell = table.cell(0, 0)
    right_cell = table.cell(0, 1)
    
    # Add property image to left cell
    left_paragraph = left_cell.paragraphs[0]
    left_run = left_paragraph.add_run()
    left_run.add_picture(property_image_path, width=Inches(4.3))
    
    # Set right border for left cell
    set_cell_border(left_cell)
    
    # Add property details to right cell
    # Location
    location_title = right_cell.paragraphs[0].add_run('Ubicación de la propiedad\n')
    location_title.font.bold = True
    location_title.font.size = Pt(14)
    
    location_text = right_cell.paragraphs[0].add_run(f"{property_data['ubicacion']}\n\n")
    location_text.font.size = Pt(12)
    
    # Value
    value_title = right_cell.paragraphs[0].add_run('Valor de la Propiedad\n')
    value_title.font.bold = True
    value_title.font.size = Pt(14)
    
    value_text = right_cell.paragraphs[0].add_run(f"${property_data['valor']:,}\n\n")
    value_text.font.size = Pt(12)
    
    # Date
    date_title = right_cell.paragraphs[0].add_run('Fecha de entrega\n')
    date_title.font.bold = True
    date_title.font.size = Pt(14)
    
    date_text = right_cell.paragraphs[0].add_run(property_data['fecha'])
    date_text.font.size = Pt(12)
    
    # Clear other cell borders
    for cell in table._cells:
        tc = cell._tc
        tcPr = tc.get_or_add_tcPr()
        tcBorders = OxmlElement('w:tcBorders')
        for border in ['top', 'left', 'bottom']:
            element = OxmlElement(f'w:{border}')
            element.set(qn('w:val'), 'none')
            tcBorders.append(element)
        # Remove existing borders
        for border in tcPr.findall('.//w:tcBorders'):
            tcPr.remove(border)
        tcPr.append(tcBorders)
    
    return document

# Example usage
if __name__ == "__main__":
    property_data = {
        'ubicacion': 'Circuito El Cerrito 140, Privadas de La Herradura, CP 42082, Pachuca de Soto, Hidalgo.',
        'valor': 3500000,
        'fecha': '26/06/2024'
    }
    
    doc = create_property_document(
        'tools/DatLogo.png',
        'tools/Propiedad.jpg',
        property_data
    )
    
    doc.save('Explorador.docx')

SyntaxError: prefix 'w' not found in prefix map (<string>)

In [5]:

from fpdf import Template

#this will define the ELEMENTS that will compose the template. 
elements = [
    { 'name': 'company_logo', 'type': 'I', 'x1': 20.0, 'y1': 17.0, 'x2': 78.0, 'y2': 30.0, 'font': None, 'size': 0.0, 'bold': 0, 'italic': 0, 'underline': 0, 'foreground': 0, 'background': 0, 'align': 'I', 'text': 'logo', 'priority': 2, },
    { 'name': 'company_name', 'type': 'T', 'x1': 17.0, 'y1': 32.5, 'x2': 115.0, 'y2': 37.5, 'font': 'Arial', 'size': 12.0, 'bold': 1, 'italic': 0, 'underline': 0, 'foreground': 0, 'background': 0, 'align': 'I', 'text': '', 'priority': 2, },
    { 'name': 'box', 'type': 'B', 'x1': 15.0, 'y1': 15.0, 'x2': 185.0, 'y2': 260.0, 'font': 'Arial', 'size': 0.0, 'bold': 0, 'italic': 0, 'underline': 0, 'foreground': 0, 'background': 0, 'align': 'I', 'text': None, 'priority': 0, },
    { 'name': 'box_x', 'type': 'B', 'x1': 95.0, 'y1': 15.0, 'x2': 105.0, 'y2': 25.0, 'font': 'Arial', 'size': 0.0, 'bold': 1, 'italic': 0, 'underline': 0, 'foreground': 0, 'background': 0, 'align': 'I', 'text': None, 'priority': 2, },
    { 'name': 'line1', 'type': 'L', 'x1': 100.0, 'y1': 25.0, 'x2': 100.0, 'y2': 57.0, 'font': 'Arial', 'size': 0, 'bold': 0, 'italic': 0, 'underline': 0, 'foreground': 0, 'background': 0, 'align': 'I', 'text': None, 'priority': 3, },
    { 'name': 'barcode', 'type': 'BC', 'x1': 20.0, 'y1': 246.5, 'x2': 140.0, 'y2': 254.0, 'font': 'Interleaved 2of5 NT', 'size': 0.75, 'bold': 0, 'italic': 0, 'underline': 0, 'foreground': 0, 'background': 0, 'align': 'I', 'text': '200000000001000159053338016581200810081', 'priority': 3, },
]

#here we instantiate the template and define the HEADER
f = Template(format="A4", elements=elements,
             title="Sample Invoice")
f.add_page()

#we FILL some of the fields of the template with the information we want
#note we access the elements treating the template instance as a "dict"
f["company_name"] = "Sample Company"
f["company_logo"] = "D:/yoe11/DOCS/MINE/Personal/Designs/Y7K11/CREATIONS/POST/2nd_Brain_Notion #1/2nd Brain_NOTION_COVER.png"
#"D:\yoe11\DOCS\MINE\Personal\Designs\Y7K11\CREATIONS\POST\2nd_Brain_Notion #1\2nd Brain_NOTION_COVER.png"
#and now we render the page
f.render("./template.pdf")

''